In [157]:
from utils.BackTest import Engine, Strategy
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [158]:
df_1 = pd.read_csv('../data/processed/merged_df.csv')
df_2 = pd.read_csv('../data/processed/features_trading_strategy.csv')

In [159]:
df_1

,datetime,EMA_8,EMA_13,EMA_21,EMA_signal,EMA_short,EMA_long,MACD,Signal_Line,MACD_Hist,...,OBV_Signal,ATR,reddit_smoothed_sentiment_weibull,return,hourly_volatility,TOTAL_TRADES,VOLUME,smoothed_sentiment,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull
0,2024-03-02 02:00:00+00:00,62220.598138,62172.294796,62076.132135,1,62184.157168,62011.323303,172.833864,187.562435,-14.728570,...,1,384.756667,0.157746,0.001309,0.006053,56088,1153.46042,-222.074588,-188.470966,0.000000
1,2024-03-02 03:00:00+00:00,62239.898552,62191.602682,62097.161032,1,62203.125296,62033.258614,169.866682,184.023284,-14.156603,...,0,347.451667,0.092267,0.003160,0.004931,67185,1465.47878,-204.266170,-204.409405,0.552411
2,2024-03-02 04:00:00+00:00,62194.576652,62169.366585,62091.596392,1,62177.406020,62033.457976,143.948043,176.008236,-32.060193,...,-1,333.268333,0.050600,-0.004357,0.005180,56098,1147.71004,-234.015147,-205.344729,0.000000
3,2024-03-02 05:00:00+00:00,62152.792951,62146.107073,62083.864902,1,62151.120478,62031.464793,119.655685,164.737726,-45.082040,...,0,326.113333,0.026060,-0.000474,0.004688,65329,1415.65823,-225.408115,-200.706776,0.000000
4,2024-03-02 06:00:00+00:00,62106.838962,62117.520348,62071.331729,0,62119.563482,62025.134067,94.429414,150.676063,-56.246649,...,0,325.211667,0.012619,-0.000977,0.004189,57826,1059.53378,-208.216550,-185.091972,1.010990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8376,2025-01-26 07:00:00+00:00,104968.753394,104930.387351,104892.740104,1,104936.968508,104874.604967,62.363541,41.106721,21.256820,...,1,298.715000,0.055311,0.000072,0.001646,41775,208.35672,30.142278,-13.564028,0.000000
8377,2025-01-26 08:00:00+00:00,104982.452640,104944.674872,104905.254640,1,104951.342584,104886.145340,65.197244,45.924826,19.272419,...,0,325.368333,0.030398,0.000305,0.001628,63281,283.50224,35.082025,-6.781369,0.103237
8378,2025-01-26 09:00:00+00:00,104904.136498,104899.725605,104880.234218,1,104901.909879,104867.173833,34.736046,43.687070,-8.951024,...,-1,326.935000,0.015620,-0.003812,0.001915,91103,621.23723,37.058349,0.681248,0.000000
8379,2025-01-26 10:00:00+00:00,104862.170609,104873.377661,104865.239289,-1,104873.199128,104855.923179,17.275949,38.404846,-21.128896,...,1,277.221667,0.007520,0.000815,0.001946,50517,252.60356,33.197390,8.190720,0.000000


In [160]:
df_1 = df_1.drop_duplicates(subset=['datetime'], keep='first')

In [161]:
df_1

,datetime,EMA_8,EMA_13,EMA_21,EMA_signal,EMA_short,EMA_long,MACD,Signal_Line,MACD_Hist,...,OBV_Signal,ATR,reddit_smoothed_sentiment_weibull,return,hourly_volatility,TOTAL_TRADES,VOLUME,smoothed_sentiment,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull
0,2024-03-02 02:00:00+00:00,62220.598138,62172.294796,62076.132135,1,62184.157168,62011.323303,172.833864,187.562435,-14.728570,...,1,384.756667,0.157746,0.001309,0.006053,56088,1153.46042,-222.074588,-188.470966,0.000000
1,2024-03-02 03:00:00+00:00,62239.898552,62191.602682,62097.161032,1,62203.125296,62033.258614,169.866682,184.023284,-14.156603,...,0,347.451667,0.092267,0.003160,0.004931,67185,1465.47878,-204.266170,-204.409405,0.552411
2,2024-03-02 04:00:00+00:00,62194.576652,62169.366585,62091.596392,1,62177.406020,62033.457976,143.948043,176.008236,-32.060193,...,-1,333.268333,0.050600,-0.004357,0.005180,56098,1147.71004,-234.015147,-205.344729,0.000000
3,2024-03-02 05:00:00+00:00,62152.792951,62146.107073,62083.864902,1,62151.120478,62031.464793,119.655685,164.737726,-45.082040,...,0,326.113333,0.026060,-0.000474,0.004688,65329,1415.65823,-225.408115,-200.706776,0.000000
4,2024-03-02 06:00:00+00:00,62106.838962,62117.520348,62071.331729,0,62119.563482,62025.134067,94.429414,150.676063,-56.246649,...,0,325.211667,0.012619,-0.000977,0.004189,57826,1059.53378,-208.216550,-185.091972,1.010990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8376,2025-01-26 07:00:00+00:00,104968.753394,104930.387351,104892.740104,1,104936.968508,104874.604967,62.363541,41.106721,21.256820,...,1,298.715000,0.055311,0.000072,0.001646,41775,208.35672,30.142278,-13.564028,0.000000
8377,2025-01-26 08:00:00+00:00,104982.452640,104944.674872,104905.254640,1,104951.342584,104886.145340,65.197244,45.924826,19.272419,...,0,325.368333,0.030398,0.000305,0.001628,63281,283.50224,35.082025,-6.781369,0.103237
8378,2025-01-26 09:00:00+00:00,104904.136498,104899.725605,104880.234218,1,104901.909879,104867.173833,34.736046,43.687070,-8.951024,...,-1,326.935000,0.015620,-0.003812,0.001915,91103,621.23723,37.058349,0.681248,0.000000
8379,2025-01-26 10:00:00+00:00,104862.170609,104873.377661,104865.239289,-1,104873.199128,104855.923179,17.275949,38.404846,-21.128896,...,1,277.221667,0.007520,0.000815,0.001946,50517,252.60356,33.197390,8.190720,0.000000


In [162]:
df_2

,datetime,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,TOTAL_TRADES_BUY,TOTAL_TRADES_SELL,VOLUME,VOLUME_BUY,...,%D,Stochastic_Signal,OBV,OBV_EMA20,OBV_Signal,H-L,H-PC,L-PC,TR,ATR
0,2024-02-22 19:00:00+00:00,51718.81,51718.82,51460.80,51509.99,50652,27901,22751,1412.76654,602.38295,...,NaN,0,NaN,NaN,0,258.02,NaN,NaN,258.02,NaN
1,2024-02-22 20:00:00+00:00,51509.99,52065.78,51461.10,51995.58,108926,71626,37300,3121.92839,1779.83133,...,NaN,0,3121.92839,3121.928390,0,604.68,555.79,48.89,604.68,NaN
2,2024-02-22 21:00:00+00:00,51995.58,52007.65,51545.26,51656.96,54274,25981,28293,1264.97024,667.58788,...,NaN,0,-1264.97024,2704.128520,-1,462.39,12.07,450.32,462.39,NaN
3,2024-02-22 22:00:00+00:00,51656.96,51717.87,51184.04,51456.01,73657,24349,49308,1357.06045,695.54171,...,NaN,0,-1357.06045,2317.348619,0,533.83,60.91,472.92,533.83,NaN
4,2024-02-22 23:00:00+00:00,51456.01,51482.69,51230.00,51288.42,38825,16468,22357,1038.76395,535.06306,...,NaN,0,-1038.76395,1997.718850,0,252.69,26.68,226.01,252.69,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2025-03-03 14:00:00+00:00,93586.83,93721.37,89136.45,89278.88,909864,449798,460066,6172.59027,2600.65274,...,51.833151,0,-6172.59027,91.667872,-1,4584.92,134.54,4450.38,4584.92,1402.496667
8996,2025-03-03 15:00:00+00:00,89278.88,91307.00,89117.00,90035.37,815439,415993,399446,5771.66863,2921.20810,...,39.186775,0,5771.66863,632.620325,1,2190.00,2028.12,161.88,2190.00,1618.750000
8997,2025-03-03 16:00:00+00:00,90035.37,90750.91,89691.02,90154.00,448660,224630,224030,2111.23659,1038.58417,...,12.624798,0,2111.23659,773.440922,0,1059.89,715.54,344.35,1059.89,1622.873333
8998,2025-03-03 17:00:00+00:00,90154.00,91000.00,89912.00,90300.01,331538,185179,146359,1626.75033,767.44993,...,20.525927,0,1626.75033,854.708485,0,1088.00,846.00,242.00,1088.00,1720.470000


In [163]:
df_3 = df_1[['datetime','reddit_smoothed_sentiment_weibull','news_smoothed_sentiment_weibull','tele_smoothed_sentiment_weibull']]

In [164]:
df_3[1:].describe()

,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull
count,7.929000e+03,7929.000000,7929.000000
mean,2.994162e-02,-105.388053,0.304112
std,8.777959e-01,395.040101,0.822927
min,-4.277553e+00,-5128.788745,-3.930538
25%,-3.054081e-01,-257.068523,0.000000
50%,-9.775172e-18,-70.549400,0.000000
75%,2.992846e-01,99.450326,0.000000
max,5.323996e+00,1046.597034,7.955905


In [165]:
df_4 = pd.merge(df_3, df_2, on='datetime', how='inner')

In [166]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7930 entries, 0 to 7929
Data columns (total 48 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   datetime                           7930 non-null   object 
 1   reddit_smoothed_sentiment_weibull  7930 non-null   float64
 2   news_smoothed_sentiment_weibull    7930 non-null   float64
 3   tele_smoothed_sentiment_weibull    7930 non-null   float64
 4   OPEN                               7930 non-null   float64
 5   HIGH                               7930 non-null   float64
 6   LOW                                7930 non-null   float64
 7   CLOSE                              7930 non-null   float64
 8   TOTAL_TRADES                       7930 non-null   int64  
 9   TOTAL_TRADES_BUY                   7930 non-null   int64  
 10  TOTAL_TRADES_SELL                  7930 non-null   int64  
 11  VOLUME                             7930 non-null   float

In [167]:
#df_4.to_csv(r'../data/processed/combined_df.csv')

In [168]:
df_4 = pd.read_csv(r'../data/processed/combined_df_with_volatility.csv')

In [169]:
df_4

,Unnamed: 0,datetime,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,...,H-L,H-PC,L-PC,TR,ATR,log_return_100,predicted_conditional_volatility,rolling_mean_volatility,rolling_std_volatility,volatility_category
0,520,2024-03-23 18:00:00+00:00,-0.000662,232.686671,0.000000,65652.70,65681.72,65270.01,65373.88,56859,...,411.71,29.02,382.69,411.71,658.445000,-0.425594,0.005914,0.006518,0.001199,Normal
1,521,2024-03-23 19:00:00+00:00,-0.000241,243.310915,0.000000,65373.88,65485.15,64786.25,64872.90,73300,...,698.90,111.27,587.63,698.90,701.053333,-0.769282,0.005821,0.006418,0.001168,Normal
2,522,2024-03-23 20:00:00+00:00,-0.000083,263.874155,0.000000,64872.90,65118.20,64672.93,65009.78,55912,...,445.27,245.30,199.97,445.27,604.058333,0.210775,0.006339,0.006296,0.001028,Normal
3,523,2024-03-23 21:00:00+00:00,-0.000027,265.004313,0.000000,65009.78,65199.97,64780.00,64944.69,45367,...,419.97,190.19,229.78,419.97,571.245000,-0.100174,0.005951,0.006184,0.000926,Normal
4,524,2024-03-23 22:00:00+00:00,-0.000008,248.893742,0.000000,64944.69,65050.72,64744.55,64747.86,46105,...,306.17,106.03,200.14,306.17,468.430000,-0.303533,0.005584,0.006049,0.000790,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,7925,2025-01-26 07:00:00+00:00,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,...,150.00,109.25,40.75,150.00,298.715000,0.007238,0.003691,0.003776,0.000097,Normal
7406,7926,2025-01-26 08:00:00+00:00,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,...,358.05,223.01,135.04,358.05,325.368333,0.030520,0.003626,0.003759,0.000091,Low
7407,7927,2025-01-26 09:00:00+00:00,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,...,586.40,0.01,586.39,586.40,326.935000,-0.381923,0.003572,0.003744,0.000097,Low
7408,7928,2025-01-26 10:00:00+00:00,0.007520,8.190720,0.000000,104630.03,104722.90,104500.00,104715.29,50517,...,222.90,92.87,130.03,222.90,277.221667,0.081454,0.003957,0.003754,0.000108,High


In [170]:
def custom_minmax_zero_anchor(series):
    x = series.copy()
    max_pos = x[x > 0].max()
    min_neg = x[x < 0].min()

    # Scale positive values
    if max_pos != 0:
        x[x > 0] = x[x > 0] / max_pos
    else:
        x[x > 0] = 0

    # Scale negative values
    if min_neg != 0:
        neg_scaled = x[x < 0] / abs(min_neg)
        x[x < 0] = neg_scaled  # values are negative, because x[x < 0] and min_neg are both negative
    else:
        x[x < 0] = 0

    return x

In [171]:
cols = [
    'reddit_smoothed_sentiment_weibull',
    'news_smoothed_sentiment_weibull',
    'tele_smoothed_sentiment_weibull'
]

# Apply to each column (column-wise)
df_4_scaled = df_4.copy()

# Normalize each column individually
normalized = df_4[cols].apply(custom_minmax_zero_anchor)

# Assign normalized columns with new names
normalized.columns = [col + '_normalized' for col in cols]
df_4_scaled = pd.concat([df_4_scaled, normalized], axis=1)

In [172]:
df_4_scaled

,Unnamed: 0,datetime,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,...,TR,ATR,log_return_100,predicted_conditional_volatility,rolling_mean_volatility,rolling_std_volatility,volatility_category,reddit_smoothed_sentiment_weibull_normalized,news_smoothed_sentiment_weibull_normalized,tele_smoothed_sentiment_weibull_normalized
0,520,2024-03-23 18:00:00+00:00,-0.000662,232.686671,0.000000,65652.70,65681.72,65270.01,65373.88,56859,...,411.71,658.445000,-0.425594,0.005914,0.006518,0.001199,Normal,-0.000155,0.245980,0.00000
1,521,2024-03-23 19:00:00+00:00,-0.000241,243.310915,0.000000,65373.88,65485.15,64786.25,64872.90,73300,...,698.90,701.053333,-0.769282,0.005821,0.006418,0.001168,Normal,-0.000056,0.257211,0.00000
2,522,2024-03-23 20:00:00+00:00,-0.000083,263.874155,0.000000,64872.90,65118.20,64672.93,65009.78,55912,...,445.27,604.058333,0.210775,0.006339,0.006296,0.001028,Normal,-0.000019,0.278949,0.00000
3,523,2024-03-23 21:00:00+00:00,-0.000027,265.004313,0.000000,65009.78,65199.97,64780.00,64944.69,45367,...,419.97,571.245000,-0.100174,0.005951,0.006184,0.000926,Normal,-0.000006,0.280144,0.00000
4,524,2024-03-23 22:00:00+00:00,-0.000008,248.893742,0.000000,64944.69,65050.72,64744.55,64747.86,46105,...,306.17,468.430000,-0.303533,0.005584,0.006049,0.000790,Normal,-0.000002,0.263113,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,7925,2025-01-26 07:00:00+00:00,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,...,150.00,298.715000,0.007238,0.003691,0.003776,0.000097,Normal,0.010389,-0.002645,0.00000
7406,7926,2025-01-26 08:00:00+00:00,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,...,358.05,325.368333,0.030520,0.003626,0.003759,0.000091,Low,0.005710,-0.001322,0.01439
7407,7927,2025-01-26 09:00:00+00:00,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,...,586.40,326.935000,-0.381923,0.003572,0.003744,0.000097,Low,0.002934,0.000720,0.00000
7408,7928,2025-01-26 10:00:00+00:00,0.007520,8.190720,0.000000,104630.03,104722.90,104500.00,104715.29,50517,...,222.90,277.221667,0.081454,0.003957,0.003754,0.000108,High,0.001412,0.008659,0.00000


In [173]:
# # Specify target range
# scaler = MinMaxScaler(feature_range=(-1, 1))

# df_4[['reddit_smoothed_sentiment_weibull_normalized','news_smoothed_sentiment_weibull_normalized','tele_smoothed_sentiment_weibull_normalized']] = scaler.fit_transform(df_4[['reddit_smoothed_sentiment_weibull','news_smoothed_sentiment_weibull','tele_smoothed_sentiment_weibull']])

In [174]:
#VWAP strategy
def determine_signal(df):
    """
    Determine the trading signal based on the strategy's criteria.
    """
    if df['CLOSE'] > df['VWAP']:
        return 1
    elif df['CLOSE'] < df['VWAP']:
        return 0
    else:
        return None

def calculate_vwap(df):
    """
    Calculate the VWAP (Volume Weighted Average Price) for each bar in the data.
    """
    df['VWAP'] = (df['CLOSE'] * df['VOLUME']).cumsum() / df['VOLUME'].cumsum()



In [175]:
df_4_scaled['VWAP'] = (df_4_scaled['CLOSE'] * df_4_scaled['VOLUME']).cumsum() / df_4_scaled['VOLUME'].cumsum()

In [176]:
df_4_scaled['VWAP_signal'] = df_4_scaled.apply(determine_signal, axis=1)

In [177]:
#retrieve all columns with the substring '_signal'
signal_cols = [col for col in df_4_scaled.columns if col.lower().endswith('_signal')]
sentinment_cols = [col for col in df_4_scaled.columns if col.lower().endswith('_sentiment_weibull_normalized')]

In [178]:
signal_cols

['EMA_Signal',
 'MACD_Signal',
 'BB_RSI_Signal',
 'RSI_Signal',
 'GoldenCross_Signal',
 'Stochastic_Signal',
 'OBV_Signal',
 'VWAP_signal']

In [179]:
sentinment_cols

['reddit_smoothed_sentiment_weibull_normalized',
 'news_smoothed_sentiment_weibull_normalized',
 'tele_smoothed_sentiment_weibull_normalized']

In [180]:
df_4_scaled[sentinment_cols].describe()

,reddit_smoothed_sentiment_weibull_normalized,news_smoothed_sentiment_weibull_normalized,tele_smoothed_sentiment_weibull_normalized
count,7.410000e+03,7410.000000,7410.000000
mean,-6.308566e-03,0.047153,0.037859
std,1.822304e-01,0.184684,0.114002
min,-1.000000e+00,-1.000000,-1.000000
25%,-7.052906e-02,-0.049384,0.000000
50%,-8.403643e-19,-0.013476,0.000000
75%,5.513989e-02,0.107465,0.000000
max,1.000000e+00,1.000000,1.000000


In [181]:
combined_strategy_cols = signal_cols + sentinment_cols

In [182]:
strategy_dict = {col: 0 for col in combined_strategy_cols}

In [183]:
strategy_dict

{'EMA_Signal': 0,
 'MACD_Signal': 0,
 'BB_RSI_Signal': 0,
 'RSI_Signal': 0,
 'GoldenCross_Signal': 0,
 'Stochastic_Signal': 0,
 'OBV_Signal': 0,
 'VWAP_signal': 0,
 'reddit_smoothed_sentiment_weibull_normalized': 0,
 'news_smoothed_sentiment_weibull_normalized': 0,
 'tele_smoothed_sentiment_weibull_normalized': 0}

In [184]:
for name in strategy_dict.keys():
    if name in sentinment_cols:
        strategy_dict[name] = 1/len(sentinment_cols)

In [185]:
strategy_dict

{'EMA_Signal': 0,
 'MACD_Signal': 0,
 'BB_RSI_Signal': 0,
 'RSI_Signal': 0,
 'GoldenCross_Signal': 0,
 'Stochastic_Signal': 0,
 'OBV_Signal': 0,
 'VWAP_signal': 0,
 'reddit_smoothed_sentiment_weibull_normalized': 0.3333333333333333,
 'news_smoothed_sentiment_weibull_normalized': 0.3333333333333333,
 'tele_smoothed_sentiment_weibull_normalized': 0.3333333333333333}

In [186]:
strategy_dict = {
    'RSI_Signal': 0.00,
    'MACD_Signal': 0.00,
    'BB_RSI_Signal': 0.00,
    'EMA_Signal': 1.00,
    'GoldenCross_Signal': 0.00,
    'Stochastic_Signal': 0.00,
    'OBV_Signal': 0.00,
    'VWAP_signal': 0.00,
    'tele_smoothed_sentiment_weibull_normalized': 0.00,
    'reddit_smoothed_sentiment_weibull_normalized': 0.00,
    'news_smoothed_sentiment_weibull_normalized': 0.00,
}

In [187]:
df_4_scaled['datetime'] = pd.to_datetime(df_4_scaled['datetime'])

In [188]:
df_filtered =df_4_scaled.dropna().set_index('datetime')

In [189]:
df_filtered

,Unnamed: 0,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,TOTAL_TRADES_BUY,...,log_return_100,predicted_conditional_volatility,rolling_mean_volatility,rolling_std_volatility,volatility_category,reddit_smoothed_sentiment_weibull_normalized,news_smoothed_sentiment_weibull_normalized,tele_smoothed_sentiment_weibull_normalized,VWAP,VWAP_signal
datetime,,,,,,,,,,,,,,,,,,,,,
2024-03-23 18:00:00+00:00,520,-0.000662,232.686671,0.000000,65652.70,65681.72,65270.01,65373.88,56859,25413,...,-0.425594,0.005914,0.006518,0.001199,Normal,-0.000155,0.245980,0.00000,65373.880000,1
2024-03-23 19:00:00+00:00,521,-0.000241,243.310915,0.000000,65373.88,65485.15,64786.25,64872.90,73300,32361,...,-0.769282,0.005821,0.006418,0.001168,Normal,-0.000056,0.257211,0.00000,65071.095189,0
2024-03-23 20:00:00+00:00,522,-0.000083,263.874155,0.000000,64872.90,65118.20,64672.93,65009.78,55912,27474,...,0.210775,0.006339,0.006296,0.001028,Normal,-0.000019,0.278949,0.00000,65053.991525,0
2024-03-23 21:00:00+00:00,523,-0.000027,265.004313,0.000000,65009.78,65199.97,64780.00,64944.69,45367,21035,...,-0.100174,0.005951,0.006184,0.000926,Normal,-0.000006,0.280144,0.00000,65037.138757,0
2024-03-23 22:00:00+00:00,524,-0.000008,248.893742,0.000000,64944.69,65050.72,64744.55,64747.86,46105,21611,...,-0.303533,0.005584,0.006049,0.000790,Normal,-0.000002,0.263113,0.00000,64996.111851,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-26 07:00:00+00:00,7925,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,22248,...,0.007238,0.003691,0.003776,0.000097,Normal,0.010389,-0.002645,0.00000,73208.632432,1
2025-01-26 08:00:00+00:00,7926,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,30294,...,0.030520,0.003626,0.003759,0.000091,Low,0.005710,-0.001322,0.01439,73209.571525,1
2025-01-26 09:00:00+00:00,7927,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,41647,...,-0.381923,0.003572,0.003744,0.000097,Low,0.002934,0.000720,0.00000,73211.603274,1


In [190]:
df_filtered = df_filtered.rename(columns={'OPEN':'Open',
                                          'CLOSE':'Close',
                                          'HIGH':'High',
                                          'LOW':'Low'})

In [191]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7410 entries, 2024-03-23 18:00:00+00:00 to 2025-01-26 11:00:00+00:00
Data columns (total 58 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Unnamed: 0                                    7410 non-null   int64  
 1   reddit_smoothed_sentiment_weibull             7410 non-null   float64
 2   news_smoothed_sentiment_weibull               7410 non-null   float64
 3   tele_smoothed_sentiment_weibull               7410 non-null   float64
 4   Open                                          7410 non-null   float64
 5   High                                          7410 non-null   float64
 6   Low                                           7410 non-null   float64
 7   Close                                         7410 non-null   float64
 8   TOTAL_TRADES                                  7410 non-null   int64  
 9   TOTAL_TRADES_BU

In [192]:
class MultiStrategyAggregator(Strategy):
    def __init__(self,weights):
        super().__init__()
        self.weights = weights # as a dictionary of name and weight

    def on_bar(self):

        weights_keys = list(self.weights.keys())

        #map the features to the weights and sum score * weight
        features = self.data.loc[self.current_idx][weights_keys]

        pred_score = sum([features[key] * self.weights[key] for key in weights_keys if features[key] >= 0])

        # #calculate scores for VWAP, reddit sentiment and news sentiment
        # vwap_score = self.data.loc[self.current_idx]['VWAP_signal'] * self.weights['VWAP_signal']
        # reddit_score = self.data.loc[self.current_idx]['reddit_smoothed_sentiment_weibull_normalized'] * self.weights['reddit_smoothed_sentiment_weibull_normalized']
        # news_score = self.data.loc[self.current_idx]['news_smoothed_sentiment_weibull_normalized'] * self.weights['news_smoothed_sentiment_weibull_normalized']
        # print(f'pred_score: {pred_score}')
        # print(f'vwap_score: {vwap_score}')
        # print(f'reddit_score: {reddit_score}')
        # print(f'news_score: {news_score}')

#        print(pred_score)

        if pred_score > 0.50:
            self.buy(
                "btc",
#                size = round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
                size = round((0.10 * self.cash)/ self.close(), 8)
            )
#            print(len(self.positions))

In [193]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7410 entries, 2024-03-23 18:00:00+00:00 to 2025-01-26 11:00:00+00:00
Data columns (total 58 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Unnamed: 0                                    7410 non-null   int64  
 1   reddit_smoothed_sentiment_weibull             7410 non-null   float64
 2   news_smoothed_sentiment_weibull               7410 non-null   float64
 3   tele_smoothed_sentiment_weibull               7410 non-null   float64
 4   Open                                          7410 non-null   float64
 5   High                                          7410 non-null   float64
 6   Low                                           7410 non-null   float64
 7   Close                                         7410 non-null   float64
 8   TOTAL_TRADES                                  7410 non-null   int64  
 9   TOTAL_TRADES_BU

In [194]:
e = Engine(asset_type='cryptocurrencies')
e.add_data(df_filtered)
e.add_strategy(MultiStrategyAggregator(strategy_dict)) # Pass both df_filtered and strategy_dict
stats = e.run()

  1%|          | 78/7410 [00:00<00:22, 321.88it/s]

2024-03-23 19:00:00+00:00 New position opened: <Position: btc size: 0.15296629 entry: 65373.88>
2024-03-23 20:00:00+00:00 New position opened: <Position: btc size: 0.15414757 entry: 64872.9>
2024-03-23 21:00:00+00:00 New position opened: <Position: btc size: 0.15382301 entry: 65009.78>
2024-03-23 22:00:00+00:00 New position opened: <Position: btc size: 0.15397718 entry: 64944.69>
2024-03-23 23:00:00+00:00 Stop Loss Filled. Limit: 64671.235 / Low: 63861.66
2024-03-23 23:00:00+00:00 Position closed for ticker btc at 64671.235
2024-03-23 23:00:00+00:00 Stop Loss Filled. Limit: 64170.255000000005 / Low: 63861.66
2024-03-23 23:00:00+00:00 Position closed for ticker btc at 64170.255000000005
2024-03-23 23:00:00+00:00 Stop Loss Filled. Limit: 64307.135 / Low: 63861.66
2024-03-23 23:00:00+00:00 Position closed for ticker btc at 64307.135
2024-03-23 23:00:00+00:00 Stop Loss Filled. Limit: 64242.045000000006 / Low: 63861.66
2024-03-23 23:00:00+00:00 Position closed for ticker btc at 64242.045000

  2%|▏         | 128/7410 [00:00<00:18, 392.38it/s]

2024-03-29 02:00:00+00:00 Cannot buy btc: maximum positions reached.
2024-03-29 03:00:00+00:00 Stop Loss Filled. Limit: 70373.84 / Low: 70274.07
2024-03-29 03:00:00+00:00 Position closed for ticker btc at 70373.84
2024-03-29 03:00:00+00:00 Stop Loss Filled. Limit: 70413.0 / Low: 70274.07
2024-03-29 03:00:00+00:00 Position closed for ticker btc at 70413.0
2024-03-29 03:00:00+00:00 Stop Loss Filled. Limit: 70451.81999999999 / Low: 70274.07
2024-03-29 03:00:00+00:00 Position closed for ticker btc at 70451.81999999999
2024-03-29 03:00:00+00:00 Stop Loss Filled. Limit: 70421.84999999999 / Low: 70274.07
2024-03-29 03:00:00+00:00 Position closed for ticker btc at 70421.84999999999
2024-03-29 04:00:00+00:00 New position opened: <Position: btc size: 0.14196032 entry: 70442.22>
2024-03-29 05:00:00+00:00 Stop Loss Filled. Limit: 70195.8425 / Low: 70176.97
2024-03-29 05:00:00+00:00 Position closed for ticker btc at 70195.8425
2024-03-29 05:00:00+00:00 Stop Loss Filled. Limit: 70235.61249999999 / L

  2%|▏         | 181/7410 [00:00<00:16, 441.82it/s]

...

 98%|█████████▊| 7245/7410 [00:15<00:00, 368.73it/s]

2025-01-15 18:00:00+00:00 New position opened: <Position: btc size: 0.10137294 entry: 98645.65>
2025-01-15 19:00:00+00:00 New position opened: <Position: btc size: 0.10055694 entry: 99446.14>
2025-01-15 20:00:00+00:00 Take Profit Filled. Limit: 100552.10833333332 / High: 100681.94
2025-01-15 20:00:00+00:00 Position closed for ticker btc at 100552.10833333332
2025-01-15 20:00:00+00:00 New position opened: <Position: btc size: 0.10015534 entry: 99844.9>
2025-01-15 21:00:00+00:00 New position opened: <Position: btc size: 0.10043549 entry: 99566.4>
2025-01-15 22:00:00+00:00 New position opened: <Position: btc size: 0.10032908 entry: 99672.0>
2025-01-15 23:00:00+00:00 New position opened: <Position: btc size: 0.10028736 entry: 99713.46>
2025-01-16 00:00:00+00:00 Take Profit Filled. Limit: 100757.24583333333 / High: 100866.66
2025-01-16 00:00:00+00:00 Position closed for ticker btc at 100757.24583333333
2025-01-16 00:00:00+00:00 Take Profit Filled. Limit: 100785.66583333333 / High: 100866.66

100%|██████████| 7410/7410 [00:15<00:00, 467.80it/s]

2025-01-22 01:00:00+00:00 Take Profit Filled. Limit: 106186.10416666667 / High: 106394.46
2025-01-22 01:00:00+00:00 Position closed for ticker btc at 106186.10416666667
2025-01-22 01:00:00+00:00 Stop Loss Filled. Limit: 105515.31749999999 / Low: 105448.99
2025-01-22 01:00:00+00:00 Position closed for ticker btc at 105515.31749999999
2025-01-22 01:00:00+00:00 Stop Loss Filled. Limit: 105685.4775 / Low: 105448.99
2025-01-22 01:00:00+00:00 Position closed for ticker btc at 105685.4775
2025-01-22 02:00:00+00:00 New position opened: <Position: btc size: 0.09451797 entry: 105799.99>
2025-01-22 03:00:00+00:00 New position opened: <Position: btc size: 0.09471142 entry: 105583.89>
2025-01-22 04:00:00+00:00 Stop Loss Filled. Limit: 105747.305 / Low: 105277.24
2025-01-22 04:00:00+00:00 Position closed for ticker btc at 105747.305
2025-01-22 04:00:00+00:00 New position opened: <Position: btc size: 0.09469696 entry: 105600.01>
2025-01-22 05:00:00+00:00 New position opened: <Position: btc size: 0.09

Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |      19.78
Average Exposure to Asset (%)            |      29.18
Strategy CAGR (%)                        |      23.85
Buy & Hold CAGR (%)                      |      74.98
Strategy Volatility (%)                  |       3.89
Buy & Hold Volatility (%)                |      10.54
Strategy Sharpe Ratio                    |       6.13
Buy & Hold Sharpe Ratio                  |       7.12
Strategy Max Drawdown (%)                |     -13.18
Buy & Hold Max Drawdown (%)              |     -31.21
Number of Trades                         |    6358.00
Number of Buys                           |    3180.00
Number of Sells                          |    3178.00
Number of Take Profits                   |    1286.00
Number of Stop Losses                    |    1892.00


In [195]:
e.plot()